In [10]:
import sys
from pathlib import Path
from zipfile import ZipFile

import pandas as pd
from bs4 import BeautifulSoup as bs

from funcs.paths import paths

In [9]:
input_dir = paths["raw_data_dir"] / "medrxiv" / "Current_Content" / "March_2023"
assert input_dir.exists(), input_dir

output_dir = paths["tmp_output"]

# folder scan

In [7]:
file_list = [_ for _ in input_dir.iterdir() if str(_).endswith(".meca")]
print(len(file_list))
print(file_list[:5])

1227
[PosixPath('/data/ik18445/projects/biorxiv-medrxiv-tdm/data/local-source-data/medrxiv/Current_Content/March_2023/b0e350bd-6d67-1014-ad19-c96e66a09703.meca'), PosixPath('/data/ik18445/projects/biorxiv-medrxiv-tdm/data/local-source-data/medrxiv/Current_Content/March_2023/de06d6bd-6c2f-1014-8025-a0d429fa0c93.meca'), PosixPath('/data/ik18445/projects/biorxiv-medrxiv-tdm/data/local-source-data/medrxiv/Current_Content/March_2023/19da261a-6c3c-1014-8618-af7a582bd05b.meca'), PosixPath('/data/ik18445/projects/biorxiv-medrxiv-tdm/data/local-source-data/medrxiv/Current_Content/March_2023/f0c7fa39-6c1b-1014-b484-8038e16734f9.meca'), PosixPath('/data/ik18445/projects/biorxiv-medrxiv-tdm/data/local-source-data/medrxiv/Current_Content/March_2023/035e37af-6bf4-1014-9da4-9c501f776874.meca')]


In [8]:
NUM_SAMPLE = 50

sample_file_list = file_list[:NUM_SAMPLE]

In [6]:
for idx, _ in enumerate(sample_file_list):
    with ZipFile(_, "r") as zip:
        print(f"#{idx}/{len(sample_file_list)}: {str(_)}")
        zip.printdir()
        print("\n\n---\n\n")

#0/50: /data/ik18445/projects/biorxiv-medrxiv-tdm/data/local-source-data/medrxiv/Current_Content/March_2023/b0e350bd-6d67-1014-ad19-c96e66a09703.meca
File Name                                             Modified             Size
content/23286782.xml                           2023-03-08 22:47:40        99666
content/23286782.pdf                           2023-03-03 15:46:16       356007
content/                                       2023-03-08 22:39:46            0
content/23286782v1_fig1.tif                    2023-03-07 02:17:30      5273524
content/23286782v1_tbl1.tif                    2023-03-07 02:17:42     13843644
content/23286782v1_tbl1a.tif                   2023-03-07 02:17:52      6487852
content/23286782v1_tbl2.tif                    2023-03-07 02:18:04     13328436
directives.xml                                 2023-03-08 23:39:50          280
manifest.xml                                   2023-03-08 23:39:48         1500
mimetype                                       202

In [ ]:
# all content scan
output_file = output_dir / "folder_scan.txt"

with output_file.open("w") as sys.stdout:
    for idx, _ in enumerate(file_list):
        with ZipFile(_, "r") as zip:
            print(f"#{idx}/{len(sample_file_list)}: {str(_)}")
            zip.printdir()
            print("\n\n---\n\n")

# zip archive scan

In [6]:
file = "manifest.xml"
with ZipFile(zip_file, "r") as zip:
    data = zip.read(file)
    print(data)

b'<?xml version="1.0" encoding="UTF-8" standalone="no"?>\r\n<!DOCTYPE manifest SYSTEM "http://schema.highwire.org/public/MECA/v0.9/Manifest/Manifest.dtd">\r\n<manifest xmlns="http://manuscriptexchange.org" version="1.0">\r\n<item type="article" id="23286782v1">\r\n<title>Epigenetic, psychological, and EEG changes after a 1-week retreat based on mindfulness and compassion for stress reduction: Study protocol of a cross-over randomized controlled trial</title>\r\n<instance media-type="application/xml" href="content/23286782.xml"/>\r\n<instance media-type="application/pdf" href="content/23286782.pdf"/>\r\n</item>\r\n<item id="directives" type="x-hw-directives">\r\n<title>HWX Processing Directives</title>\r\n<instance media-type="application/vnd.hw-ingest-pi+xml" href="directives.xml"/>\r\n</item>\r\n<item id="transfer" type="transfer-details">\r\n<title>MECA Transfer Info</title>\r\n<instance media-type="application/meca-xfer+xml" href="transfer.xml"/>\r\n</item>\r\n<item id="fig1" type="

In [7]:
bs_content = bs(data, "xml")

In [12]:
res = bs_content.find("manifest")
print(res.prettify())

<manifest version="1.0" xmlns="http://manuscriptexchange.org">
 <item id="23286782v1" type="article">
  <title>
   Epigenetic, psychological, and EEG changes after a 1-week retreat based on mindfulness and compassion for stress reduction: Study protocol of a cross-over randomized controlled trial
  </title>
  <instance href="content/23286782.xml" media-type="application/xml"/>
  <instance href="content/23286782.pdf" media-type="application/pdf"/>
 </item>
 <item id="directives" type="x-hw-directives">
  <title>
   HWX Processing Directives
  </title>
  <instance href="directives.xml" media-type="application/vnd.hw-ingest-pi+xml"/>
 </item>
 <item id="transfer" type="transfer-details">
  <title>
   MECA Transfer Info
  </title>
  <instance href="transfer.xml" media-type="application/meca-xfer+xml"/>
 </item>
 <item id="fig1" type="figure">
  <title>
   Fig 1
  </title>
  <instance href="content/23286782v1_fig1.tif" media-type="image/tiff"/>
 </item>
 <item id="tbl1" type="table">
  <tit

In [13]:
res = bs_content.find_all("instance")
print(res)

[<instance href="content/23286782.xml" media-type="application/xml"/>, <instance href="content/23286782.pdf" media-type="application/pdf"/>, <instance href="directives.xml" media-type="application/vnd.hw-ingest-pi+xml"/>, <instance href="transfer.xml" media-type="application/meca-xfer+xml"/>, <instance href="content/23286782v1_fig1.tif" media-type="image/tiff"/>, <instance href="content/23286782v1_tbl1.tif" media-type="image/tiff"/>, <instance href="content/23286782v1_tbl1a.tif" media-type="image/tiff"/>, <instance href="content/23286782v1_tbl2.tif" media-type="image/tiff"/>]


In [19]:
res[0].attrs["href"]

'content/23286782.xml'